In [ ]:
# import sys
# print(sys.executable)

In [ ]:
# from rag_public_reports.config import ROOT_DIR, DATA_DIR

# print(ROOT_DIR)   # doit afficher .../rag-rapports-publics
# print(DATA_DIR)   # doit afficher .../rag-rapports-publics/data

In [2]:
from rag_public_reports.ingest import ingest_pdf
from rag_public_reports.vectorstore import get_vector_store, add_documents, is_already_ingested
from rag_public_reports.utils import list_ingested_reports, print_chunk_sample
from rag_public_reports.rag import answer

In [ ]:
vs = get_vector_store()

🗄️  Vector store chargé — 1305 chunks en base
✅ Collection supprimée


In [3]:
from rag_public_reports.config import DATA_DIR

chunks = ingest_pdf(
    file_path=DATA_DIR / "raw" / "2025_Strategie-nationale-IA_ccomptes.pdf",
    institution="Cour des comptes",
    year=2025,
    title="La stratégie nationale pour l'IA",
    theme="numérique",
    strategy="sections",
)

vs = get_vector_store()
doc_ids = add_documents(vs, chunks)

📄  Chargement : 2025_Strategie-nationale-IA_ccomptes.pdf  [stratégie : sections]
    → 308206 caractères chargés
✅  209 chunks créés
    → Longueur moyenne : 1614 caractères
    → Sections détectées : 209 chunks avec titre de section
🗄️  Vector store chargé — 0 chunks en base
➕ 209 chunks ajoutés au vector store


In [ ]:
vs = get_vector_store()

if not is_already_ingested(vs, str(DATA_DIR / "raw" / "2025_Strategie-nationale-IA_ccomptes.pdf")):
    doc_ids = add_documents(vs, chunks)
else:
    print("Déjà indexé !")

In [ ]:
list_ingested_reports(vs)

In [4]:
print_chunk_sample("recommandations sur la stratégie nationale IA", vs)


🔍 Top 3 chunks pour : "recommandations sur la stratégie nationale IA"

─── Chunk 1 ───────────────────────────────────────────
Source : Cour des comptes 2025 — La stratégie nationale pour l'IA
Section : SYNTHÈSE

SYNTHÈSE 
 
 
11 
C’est à cette condition que la France, en bonne intelligence avec l’Union européenne et 
les collectivités locales , en s’appuyant sur l’écosystème de formation -recherche-innovation 
comme sur les entreprises et les investisseurs, quitte à modifier ses modes opératoires, 
continuera à s’inscrire dans une trajectoire d’excellence et parviendra à embrasser l’ensemble 
des dimensions que la révolution de l’IA est appelée à toucher, au service du bien commun et 
dans un souci de g

─── Chunk 2 ───────────────────────────────────────────
Source : Cour des comptes 2025 — La stratégie nationale pour l'IA
Section : COUR DES COMPTES

dans tous les secteurs de la recherche, réorienter l’effort de formation continue, et 
accompagner les mutations professionnelles liée

In [ ]:
from rag_public_reports.rag import answer
print(answer(
    "Quelles sont les principales recommandations de la Cour des comptes sur la stratégie nationale pour l'IA ?",
    filter_institution="Cour des comptes",
    verbose=True
))

In [ ]:
# Diagnostic — affiche les titres détectés chunk par chunk
for i, chunk in enumerate(chunks):
    section = chunk.metadata.get("section", "")
    if any(mot in section.lower() for mot in ["recommand", "recap", "synthes", "conclusion"]):
        print(f"Chunk {i:3d} — section : '{section}'")
        print(f"           début : {chunk.page_content[:100]}")
        print()

In [ ]:
# Recharge le PDF page par page pour inspecter
from langchain_community.document_loaders import PyPDFLoader
from rag_public_reports.config import DATA_DIR

loader = PyPDFLoader(str(DATA_DIR / "raw" / "2025_Strategie-nationale-IA_ccomptes.pdf"))
pages = loader.load()

# Affiche les pages 11 à 14 (0-indexé)
for i in range(11, 15):
    print(f"\n{'='*60}")
    print(f"PAGE {i+1}")
    print('='*60)
    print(pages[i].page_content[:500])

In [ ]:
from rag_public_reports.rag import answer
print(answer(
    "Quelles sont les principales critiques adressées à l'Etat sur la mise en oeuvre de cette stratégie ?",
    filter_institution="Cour des comptes",
    verbose=True
))

In [ ]:
# # Vérifier la distribution des tailles de chunks
# import matplotlib.pyplot as plt

# sizes = [len(c.page_content) for c in chunks]
# print(f"Min    : {min(sizes)} caractères")
# print(f"Max    : {max(sizes)} caractères")
# print(f"Moyenne: {sum(sizes)//len(sizes)} caractères")

# plt.hist(sizes, bins=30)
# plt.xlabel("Taille du chunk (caractères)")
# plt.title("Distribution des chunks")
# plt.show()

In [ ]:
# # Inspecter les chunks trop courts
# petits_chunks = [c for c in chunks if len(c.page_content) < 200]
# print(f"{len(petits_chunks)} chunks de moins de 200 caractères\n")

# for c in petits_chunks[:10]:
#     print(f"[{len(c.page_content)} chars] section='{c.metadata.get('section','')}' ")
#     print(repr(c.page_content[:100]))
#     print()

In [7]:
print_chunk_sample("transformation numérique et intelligence artificielle", vs, k=5)


🔍 Top 5 chunks pour : "transformation numérique et intelligence artificielle"

─── Chunk 1 ───────────────────────────────────────────
Source : Cour des comptes 2025 — La stratégie nationale pour l'IA
Section : C - La transformation de l’action publique par l’IA, qui n’a pas été une priorité de la deuxième phase

C - La transformation de l’action publique par l’IA, qui n’a pas été une priorité de la deuxième phase  
de la SNIA, est restée très modeste ........................................................................................................................ 61

─── Chunk 2 ───────────────────────────────────────────
Source : IGF 2023 — 2023_RH-Etat-numerique_IGF
Section : 2.6. Une sensibilisation de tous les cadres aux enjeux de la transformation

2.6. Une sensibilisation de tous les cadres aux enjeux de la transformation 
numérique est nécessaire .......................................................................................................... 45 
CONCLUSION ....

In [4]:
titre = "La transformation de l'action publique par l'IA"

notes = """
Les administrations tardent à adopter l'IA malgré les injonctions.
Quelques exemples prometteurs : détection de fraude fiscale, pseudonymisation
des décisions de justice. Frein principal : manque de compétences internes
et absence de budget dédié.
"""

print(answer(
    titre,
    notes=notes,
    filter_theme="numérique",
    mode="redaction",
    verbose=False
))

🗄️  Vector store chargé — 1305 chunks en base
🔍 15 chunks récupérés
# La transformation de l'action publique par l'IA

## 1. Un déploiement encore timide malgré des injonctions répétées

L'intelligence artificielle fait l'objet, depuis l'adoption de la stratégie nationale de mars 2018, d'injonctions réitérées à l'adresse des administrations publiques. Le règlement européen du 13 juin 2024 établissant des règles harmonisées en matière d'IA est venu renforcer ce cadre normatif, imposant de nouvelles obligations aux acteurs publics. Pourtant, la réalité du déploiement demeure en deçà des ambitions affichées.

La Cour des comptes relevait ainsi, dans son rapport de 2024 sur le pilotage de la transformation numérique par la DINUM, qu'Etalab, bien que positionné aux avant-gardes du développement de l'IA pour le service public en 2019-2020, voyait ses programmes « rester éparpillés » et ses projets « peiner à passer à l'échelle ». Le programme « 10 % », lancé en 2022 sur la base du volontaria